In [70]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


In [76]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [83]:
encoder = LabelEncoder()
final_vectors = user_vectors.copy()
x = encoder.fit(final_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
username_enc

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc

#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])
final_vectors

,username,experience,gender,generation,fav_anime_period,Vampire,Supernatural,Parody,Samurai,Space,...,Arts,Game,Seinen,Slice,Fantasy,Police,Historical,Romance,Mecha,Martial
0,7537,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1179,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6556,3,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,5037,3,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3575,3,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9297,8352,3,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9298,2558,3,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9299,4967,3,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9300,9113,2,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Compute K-Nearest Neighbours

In [105]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours():
    user = np.array([4967,3, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    nc = NearestNeighbors(n_neighbors = 10, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data
    n = train.kneighbors([user], return_distance = False)
    return n

sim_names = []
neighbours = neighbours()
for i in neighbours[0]: #loop through nested array
    #print(final_arrays[i][0])
    name = x.inverse_transform([final_arrays[i][0]])
    sim_names.append(name)

#build new dataframe of neighbours
neighbour_frame = pd.DataFrame()
for i in users.columns:
    neighbour_frame[i] = None

for name in sim_names:
    user_row = users.loc[users["username"]==name[0]]
    neighbour_frame = pd.concat([neighbour_frame, user_row])
neighbour_frame


#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
9299,332815,Sicka,1912,Veteran,Male,1986,13168,Millenials,"['Comedy', 'Adventure', 'Action']",Classic
3551,58613,SsjRavi,255,Veteran,Male,1992,6600,Millenials,"['Action', 'Adventure', 'Comedy']",Classic
6085,379676,TCarneiro71,1148,Veteran,Male,1989,13127,Millenials,"['Action', 'Comedy', 'Adventure']",Classic
7187,1260091,TEX214,266,Veteran,Male,1991,7857,Millenials,"['Action', 'Comedy', 'Romance']",Classic
2802,101224,Regral,450,Veteran,Male,1987,9494,Millenials,"['Comedy', 'Adventure', 'Action']",Classic
7934,1094421,RafaKojiro,298,Veteran,Male,1988,6874,Millenials,"['Action', 'Comedy', 'Adventure']",Classic
8031,53523,_Halconnen,338,Veteran,Male,1988,7656,Millenials,"['Action', 'Comedy', 'Adventure']",Classic
4941,186845,Poitoy,48,Regular,Male,1993,1279,Millenials,"['Action', 'Comedy', 'Adventure']",Classic
8028,41781,Pliskin,132,Regular,Male,1988,2571,Millenials,"['Action', 'Adventure', 'Comedy']",Classic
9093,1200283,Purge,49,Regular,Male,1993,2387,Millenials,"['Action', 'Comedy', 'Adventure']",Classic


Create anime recommendations

In [129]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours, media_type):
    #Get top rated TV shows or movies from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        tv_shows = user_ratings[user_ratings["type"]=="TV"]
        movies = user_ratings[user_ratings["type"]=="Movie"]

        if media_type == "TV":
            show_ratings = tv_shows
        else:
            show_ratings = movies
        
        #There may be 0 movies in the user's top 10 list in which case we skip that user's recommendationan
        if len(show_ratings) > 1:
            top_rated = show_ratings.iloc[0]["title"]
            suggestions.append(top_rated)
            #print(top_rated)

    if len(suggestions)==0:
        print("No movies to recommend :/")
    
    return suggestions

recommendations(neighbour_frame, "TV")
#ratings

['Hachimitsu to Clover',
 'Tengen Toppa Gurren Lagann',
 'Dragon Ball Z',
 'Higashi no Eden',
 'Cowboy Bebop',
 'InuYasha: Kanketsu-hen',
 'Samurai Champloo',
 'Sayonara Zetsubou Sensei',
 'Neon Genesis Evangelion',
 'Gyakkyou Burai Kaiji: Ultimate Survivor']

Evaluation of the recommended anime

In [75]:
#function that returns a list of indexes of most similar users in "final_arrays" array
""" def neighbours():
    #loop through all users in test set 
    user = np.array([7537,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data from training data
    n = train.kneighbors([user], return_distance=False)
    #return nested list. each list item is a list of "final_arrays" indexes of n similar users
    return n

sim_names = []
neighbours = neighbours()
for i in neighbours[0]: #loop through nested array
    #print(final_arrays[i][0])
    name = x.inverse_transform([final_arrays[i][0]])
    sim_names.append(name)

#build new dataframe of neighbours
neighbour_frame = pd.DataFrame()
for i in users.columns:
    neighbour_frame[i] = None

for name in sim_names:
    user_row = users.loc[users["username"]==name[0]]
    neighbour_frame = pd.concat([neighbour_frame, user_row])
neighbour_frame """

#include analysis of distance metric between neighbours
#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
0,2255153,karthiga,49,Active,Female,1990,3391,Millenials,"['Comedy', 'Action', 'Shounen']",Classic
68,502880,lila94,89,Active,Female,1994,4204,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
416,2120443,hana_haruka,110,Active,Female,1993,4062,Millenials,"['Action', 'Shounen', 'Comedy']",Classic
3962,71147,gosakurago,86,Active,Female,1994,3329,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
3132,487038,gorankudasai,144,Active,Female,1992,3513,Millenials,"['Action', 'Shounen', 'Comedy']",Classic


Function that converts user information from website into vector